In [ ]:
# conda create --clone pyvizenv --name smallcapcryptosenv -y

In [ ]:
# !conda install -c anaconda seaborn -y

In [ ]:
# conda activate smallcapcryptosenv

In [ ]:
# !pip install pycoingecko

In [10]:
# Initial Imports
import pandas as pd
import os
from pathlib import Path

import requests
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [11]:
def get_date_range(delta=365):
	datetime_2 = pd.Timestamp("today").normalize() # current date as timestamp
	datetime_1 = datetime_2 - pd.to_timedelta(delta, unit="Days")
	
	dates = {
		"date_1": [datetime_1,datetime_1.timestamp()],
		"date_2": [datetime_2,datetime_2.timestamp()]}
	return dates

In [12]:
def ticker_url(token_id, date_range):
	my_url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart/range?vs_currency=usd&from={date_range['date_1'][1]}&to={date_range['date_2'][1]}"
	return my_url

In [13]:
def get_json_from_api(url):
	return requests.get(url).json()

In [14]:
crypto_list = pd.read_csv(Path("Resources/crypto_list.csv"))
crypto_list.drop(crypto_list.columns[-1],axis=1,inplace=True)

In [15]:
btc_eth = pd.read_csv(Path("Resources/btc_eth.csv"))
btc_eth.drop(btc_eth.columns[-1],axis=1,inplace=True)

In [16]:
btc_eth

,ticker,crypto_name,token_id
0,BTC,BITCOIN,bitcoin
1,ETH,ETHEREUM,ethereum


In [17]:
crypto_list.tail()

,ticker,crypto_name,token_id
38,LQTY,LIQUITY,liquity
39,BOND,BARNBRIDGE,barnbridge
40,TRB,TELLOR,tellor
41,ASM,ASSEMBLE PROTOCOL,assemble-protocol
42,NCT,POLYSWARM,polyswarm


### Coingecko API get request

In [19]:
my_dates = get_date_range() # this should only run once--keep out of loop

# my_dates['date_1'][0] # output (datetime):  Timestamp('2021-04-05 00:00:00')
# my_dates['date_1'][1] # output (unix time): 1617580800.0

In [28]:
btc_dates = get_date_range(1500)
btc_dates

{'date_1': [Timestamp('2018-02-26 00:00:00'), 1519603200.0],
 'date_2': [Timestamp('2022-04-06 00:00:00'), 1649203200.0]}

    -> Run loop of get requests for cryptos

In [23]:
# loop for small cap cryptos ('crypto_list')
i = 0
for i in range(len(crypto_list)):
	token_id = crypto_list['token_id'][i] # this should run within api call loop -- declare new token_id each api call from id_list
	url = ticker_url(token_id,my_dates) # this should run within api call loop -- new url each request
	crypto_json = get_json_from_api(url)
	
	df = pd.DataFrame.from_dict(crypto_json)
	raw_list = []
	
	j = 0
	for j in range(len(df['prices'])):
		raw = int(df['prices'][j][0]/1000)
		raw_list.append(raw)

		df['prices'][j] = df['prices'][j][1]
		df['market_caps'][j] = df['market_caps'][j][1]
		df['total_volumes'][j] = df['total_volumes'][j][1]
	
	df_dates = pd.to_datetime(raw_list, unit='s')
	
	df.insert(0,'date',df_dates)
	df.set_index('date',inplace=True)

	df['prices'] = df['prices'].astype(float, errors = 'raise')
	df['market_caps'] = df['market_caps'].astype(float, errors = 'raise')
	df['total_volumes'] = df['total_volumes'].astype(float, errors = 'raise')

	ticker_path = 'Data/' + token_id + '.csv'
	df.to_csv(ticker_path)

In [29]:
# loop for bitcoin and ethereum (running separately with expanded date range)
i = 0
for i in range(len(btc_eth)):
	token_id = btc_eth['token_id'][i] # this should run within api call loop -- declare new token_id each api call from id_list
	url = ticker_url(token_id,btc_dates) # this should run within api call loop -- new url each request
	crypto_json = get_json_from_api(url)
	
	df = pd.DataFrame.from_dict(crypto_json)
	raw_list = []
	
	j = 0
	for j in range(len(df['prices'])):
		raw = int(df['prices'][j][0]/1000)
		raw_list.append(raw)

		df['prices'][j] = df['prices'][j][1]
		df['market_caps'][j] = df['market_caps'][j][1]
		df['total_volumes'][j] = df['total_volumes'][j][1]
	
	df_dates = pd.to_datetime(raw_list, unit='s')
	
	df.insert(0,'date',df_dates)
	df.set_index('date',inplace=True)

	df['prices'] = df['prices'].astype(float, errors = 'raise')
	df['market_caps'] = df['market_caps'].astype(float, errors = 'raise')
	df['total_volumes'] = df['total_volumes'].astype(float, errors = 'raise')

	ticker_path = 'Data/' + token_id + '.csv'
	df.to_csv(ticker_path)

In [28]:
df.dtypes

prices           float64
market_caps      float64
total_volumes    float64
dtype: object